In [39]:
from utils.second_looks_utils import *
seed = 42

## Our reformatting functions
- Redundant token removal
- Example concatenation for longer training sequences
- Preposing
- Preposing + Interjection
- ReCOGS
- Variable-free

In [179]:
def translate_invariant_form(lf):
    nouns = lf.split(" AND ")[0].split(" ; ")[:-1]
    nouns_map = {}
    new_var = 0
    for noun in nouns:
        original_var = noun.split()[-2]
        new_noun = noun.replace(str(original_var), str(new_var))
        nouns_map[original_var] = new_noun
        new_var += 1
    
    conjs_set = set([])
    conjs = lf.split(" ; ")[-1].split(" AND ")
    vp_conjs_map = {}
    nested_conjs = []
    childen_count_map = {}
    for conj in conjs:
        if "nmod" in conj:
            role = conj.split()[0]
            pred = conj.split()[2]
            first_arg = conj.split()[-4]
            second_arg = conj.split()[-2]
            new_conj = f"{role} . {pred} ( {nouns_map[first_arg]} , {nouns_map[second_arg]} )"
            conjs_set.add(new_conj)
        else:
            role = conj.split()[0]
            pred = conj.split()[2]
            first_arg = conj.split()[-4]
            second_arg = conj.split()[-2]
            if first_arg == second_arg:
                continue # this is wrong, we dont decode, let error casacade
            if second_arg.isnumeric() and second_arg in nouns_map:
                second_arg = nouns_map[second_arg]
                new_conj = f"{role} . {pred} ( {second_arg} )"
                if first_arg in vp_conjs_map:
                    vp_conjs_map[first_arg].append(new_conj)
                else:
                    vp_conjs_map[first_arg] = [new_conj]
            elif second_arg.isnumeric():
                if first_arg not in childen_count_map:
                    childen_count_map[first_arg] = 1
                else:
                    childen_count_map[first_arg] += 1
                nested_conjs.append({
                    "pred": pred,
                    "role": role,
                    "first_arg": first_arg,
                    "second_arg": second_arg,
                })
            else:
                new_conj = f"{role} . {pred} ( {second_arg} )"
                if first_arg in vp_conjs_map:
                    vp_conjs_map[first_arg].append(new_conj)
                else:
                    vp_conjs_map[first_arg] = [new_conj]
                    
    print(vp_conjs_map)
    print(nested_conjs)
    print(childen_count_map)
    
    while_loop_count = 0
    while len(nested_conjs) > 0:
        while_loop_count += 1
        if while_loop_count > 100:
            break # no need, this answer has to be wrong!
        conj = nested_conjs.pop(0)
        if conj['second_arg'] not in childen_count_map or childen_count_map[conj['second_arg']] == 0:
            core = " AND ".join(vp_conjs_map[conj['second_arg']])
            vp_conjs_map[conj['first_arg']].append(f"{conj['role']} . {conj['pred']} ( {core} )")
            childen_count_map[conj['first_arg']] -= 1
        else:
            # if the conj is corrupted, then we abandon just let it go and fail to compare.
            if conj['first_arg'] == conj['second_arg']:
                continue
            nested_conjs.append(conj)
            
    print(vp_conjs_map)
    print(nested_conjs)
    print(childen_count_map)
    print(while_loop_count)
    
    filtered_conjs_set = set([])
    for k, v in vp_conjs_map.items():
        vp_conjs_map[k].sort()
    for k, v in vp_conjs_map.items():
        filtered_conjs_set.add(" AND ".join(v))
    
    return filtered_conjs_set

In [180]:
translate_invariant_form(
"girl ( 41 ) ; donut ( 18 ) ; agent . give ( 18 , 41 ) AND recipient . give ( 18 , James ) AND theme . give ( 18 , 18 )"
)

{'18': ['agent . give ( girl ( 0 ) )', 'recipient . give ( James )']}
[]
{}
{'18': ['agent . give ( girl ( 0 ) )', 'recipient . give ( James )']}
[]
{}
0


{'agent . give ( girl ( 0 ) ) AND recipient . give ( James )'}

In [167]:
translate_invariant_form(
"boy ( 23 ) ; * teacher ( 9 ) ; * cake ( 38 ) ; agent . imagine ( 47 , 23 ) AND ccomp . imagine ( 47 , 50 ) AND agent . wire ( 50 , 9 ) AND recipient . wire ( 50 , Emma ) AND theme . wire ( 50 , 38 )"
)

{'agent . imagine ( boy ( 0 ) ) AND ccomp . imagine ( agent . wire ( * teacher ( 1 ) ) AND recipient . wire ( Emma ) AND theme . wire ( * cake ( 2 ) ) )',
 'agent . wire ( * teacher ( 1 ) ) AND recipient . wire ( Emma ) AND theme . wire ( * cake ( 2 ) )'}

### Redundant Token Removal

In [2]:
for removing_set in [
    ['x', '_'],
    ['x', '_', '(', ')'],
    ['x', '_', '(', ',', ')']
]:
    def token_removal(text, phi): 
        # Parsing:
        terms = []
        for t in phi.split():
            if t not in removing_set:
                terms += [t]
        ret = " ".join(terms).strip()
        return ret
    train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

    train_df['LF'] = train_df[['sentence', 'LF']].apply(lambda x: token_removal(*x), axis=1,)
    dev_df['LF'] = dev_df[['sentence', 'LF']].apply(lambda x: token_removal(*x), axis=1,)
    test_df['LF'] = test_df[['sentence', 'LF']].apply(lambda x: token_removal(*x), axis=1,)
    gen_df['LF'] = gen_df[['sentence', 'LF']].apply(lambda x: token_removal(*x), axis=1,)
    
    removing_set_str = "".join(removing_set)
    dataset_postfix = f"remove_{removing_set_str}"
    train_df.to_csv(f'./cogs_token_removal/train_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    dev_df.to_csv(f'./cogs_token_removal/dev_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    test_df.to_csv(f'./cogs_token_removal/test_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    gen_df.to_csv(f'./cogs_token_removal/gen_{dataset_postfix}.tsv', sep='\t', index=False, header=False)

### Example Concatenations

In [152]:
def reindex(LFs, initial_indexes):
    new_LF_prefix = []
    new_LF_body = []
    for i in range(len(LFs)):
        if initial_indexes[i] != 0:
            new_lf = []
            for item in LFs[i].split():
                if item.isnumeric():
                    new_i = int(item) + initial_indexes[i]
                    new_lf += [str(new_i)]
                else:
                    new_lf += [item]
            new_lf = " ".join(new_lf)
        else:
            new_lf = LFs[i]
        
        for item in new_lf.split(" ; "):
            if "*" in item:
                new_LF_prefix += [item]
            else:
                new_LF_body += [item]
        new_LF_body += ["AND"]
    new_LF_body = new_LF_body[:-1]
    return " ; ".join(new_LF_prefix) + " ; " + " ".join(new_LF_body)

In [ ]:
append_ks = [256, 512, 1024, 2048, 3072]
for append_k in append_ks:
    train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    train_df_org = train_df.copy()
    train_df = train_df[train_df["type"] != "primitive"]
    dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    dataset_postfix = f"k_{append_k}"
    append_data = []
    start_indexes = [i*6 for i in range(append_k)]
    sorted_train_df = train_df.sort_values(by="sentence", key=lambda x: x.str.len())
    for start_index in start_indexes:
        conj_1 = sorted_train_df.iloc[-2-start_index].sentence
        if conj_1.split()[0] in {'The', 'A'}:
            conj_1_first = conj_1[0].lower()
        else:
            conj_1_first = conj_1[0]
            
        conj_2 = sorted_train_df.iloc[-3-start_index].sentence
        if conj_2.split()[0] in {'The', 'A'}:
            conj_2_first = conj_2[0].lower()
        else:
            conj_2_first = conj_2[0]
            
        append_data += [
            [sorted_train_df.iloc[-1-start_index].sentence[:-1]+\
            conj_1_first+\
            sorted_train_df.iloc[-2-start_index].sentence[1:-1]+\
            conj_2_first+\
            sorted_train_df.iloc[-3-start_index].sentence[1:],
            reindex(
                [
                    sorted_train_df.iloc[-1-start_index].LF,
                    sorted_train_df.iloc[-2-start_index].LF,
                    sorted_train_df.iloc[-3-start_index].LF
                ],
                [
                    0,
                    len(sorted_train_df.iloc[-1-start_index].sentence[:-1].strip().split()),
                    len(sorted_train_df.iloc[-1-start_index].sentence[:-1].strip().split())+
                    len(sorted_train_df.iloc[-2-start_index].sentence[:-1].strip().split())
                ]
            ),
            'concat']
        ]
    append_df = pd.DataFrame(append_data, columns =['sentence', 'LF', 'type'])
    train_df = pd.concat([train_df_org, append_df])
    train_df.to_csv(f'./cogs_concat/train_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    dev_df.to_csv(f'./cogs_concat/dev_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    test_df.to_csv(f'./cogs_concat/test_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    gen_df.to_csv(f'./cogs_concat/gen_{dataset_postfix}.tsv', sep='\t', index=False, header=False)
    
    max_s = max(train_df['sentence'].str.split().apply(len))
    max_lf = max(train_df['LF'].str.split().apply(len))
    print(max_s, max_lf)

### Preposing

In [3]:
set_seed(seed)
def translate_regular(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)

    return text, terms, _type
        
def translate(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)
    
    if "nmod" not in terms.split():
        return text, terms, _type
    
    if random.random() >= proposing_prob:
        return text, terms, _type
    
    if terms.split().count("nmod") == 2:
        upper_bound = 6
        return_type = "preposing_2"
    elif terms.split().count("nmod") == 1:
        upper_bound = 3
        return_type = "preposing_1"
    else:
        assert False
        
    nmod = terms.split()[terms.split().index("nmod")+2]
    pre_phrase = text.split()[text.split().index(nmod)-2 : text.split().index(nmod)+upper_bound]
    pre_phrase[0] = pre_phrase[0].capitalize()
    pre_text = text.split()[:text.split().index(nmod)-2]
    if pre_text[0] in ["The", "A"]:
        pre_text[0] = pre_text[0].lower()
    post_text = text.split()[text.split().index(nmod)+upper_bound:]
    pre_text = pre_phrase + pre_text + post_text
    pre_text = " ".join(pre_text)

    index_map = {}
    idx = 0
    for i in range(text.split().index(nmod)-2, text.split().index(nmod)+upper_bound):
        index_map[f"{i}"] = f"{idx}"
        idx += 1
    for i in range(text.split().index(nmod)-2):
        ii = 2+upper_bound+i
        index_map[f"{i}"] = f"{ii}"
        idx += 1
    for i in range(text.split().index(nmod)+upper_bound, len(text.split())):
        ii = idx+(i-(text.split().index(nmod)+upper_bound))
        index_map[f"{i}"] = f"{ii}"  
            
    # now handle LF
    pre_terms = []
    for t in terms.split():
        if t.isnumeric():
            pre_terms += [str(index_map[str(int(t))])]
        else:
            pre_terms += [t]
    pre_terms = " ".join(pre_terms)

    pre_terms_def = pre_terms.split(" ; ")[:-1]
    pre_terms_def.sort(key = lambda x: int(x.split()[-2]))  
    pre_terms_role = pre_terms.split(" ; ")[-1].split(" AND ")
    pre_terms_role.sort(key = lambda x: (int(x.split()[-2]) if x.split()[-5] == "(" else int(x.split()[-6]) if x.split()[-2].isnumeric() else int(x.split()[-4]), -1 if x.split()[-5] == "(" else int(x.split()[-2]) if x.split()[-2].isnumeric() else pre_text.split().index(x.split()[-2])))  
    pre_terms_role = " AND ".join(pre_terms_role)
    pre_terms = " ; ".join(pre_terms_def + [pre_terms_role])

    return pre_text, pre_terms, return_type

train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

proposing_prob = 0.05
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(lambda x: translate(*x), axis=1, result_type='expand')
dev_df[['sentence', 'LF', 'type']] = dev_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
test_df[['sentence', 'LF', 'type']] = test_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
gen_df[['sentence', 'LF', 'type']] = gen_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')

In [4]:
dataset_postfix = "preposing"
train_df.to_csv(f'./cogs_{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./cogs_{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./cogs_{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./cogs_{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)

### Preposing + Sprinkles (Interjection)

In [5]:
set_seed(seed)
def translate_regular(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)

    return text, terms, _type
        
def translate(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)
    
    if "nmod" not in terms.split():
        return text, terms, _type
    
    if random.random() >= proposing_prob:
        return text, terms, _type
    
    if terms.split().count("nmod") == 2:
        upper_bound = 6
        return_type = "preposing_2"
    elif terms.split().count("nmod") == 1:
        upper_bound = 3
        return_type = "preposing_1"
    else:
        assert False
        
    nmod = terms.split()[terms.split().index("nmod")+2]
    pre_phrase = text.split()[text.split().index(nmod)-2 : text.split().index(nmod)+upper_bound]
    pre_phrase[0] = pre_phrase[0].capitalize()
    pre_text = text.split()[:text.split().index(nmod)-2]
    if pre_text[0] in ["The", "A"]:
        pre_text[0] = pre_text[0].lower()
    post_text = text.split()[text.split().index(nmod)+upper_bound:]
    pre_text = pre_phrase + pre_text + post_text
    pre_text = " ".join(pre_text)

    index_map = {}
    idx = 0
    for i in range(text.split().index(nmod)-2, text.split().index(nmod)+upper_bound):
        index_map[f"{i}"] = f"{idx}"
        idx += 1
    for i in range(text.split().index(nmod)-2):
        ii = 2+upper_bound+i
        index_map[f"{i}"] = f"{ii}"
        idx += 1
    for i in range(text.split().index(nmod)+upper_bound, len(text.split())):
        ii = idx+(i-(text.split().index(nmod)+upper_bound))
        index_map[f"{i}"] = f"{ii}"  
            
    # now handle LF
    pre_terms = []
    for t in terms.split():
        if t.isnumeric():
            pre_terms += [str(index_map[str(int(t))])]
        else:
            pre_terms += [t]
    pre_terms = " ".join(pre_terms)

    pre_terms_def = pre_terms.split(" ; ")[:-1]
    pre_terms_def.sort(key = lambda x: int(x.split()[-2]))  
    pre_terms_role = pre_terms.split(" ; ")[-1].split(" AND ")
    pre_terms_role.sort(key = lambda x: (int(x.split()[-2]) if x.split()[-5] == "(" else int(x.split()[-6]) if x.split()[-2].isnumeric() else int(x.split()[-4]), -1 if x.split()[-5] == "(" else int(x.split()[-2]) if x.split()[-2].isnumeric() else pre_text.split().index(x.split()[-2])))  
    pre_terms_role = " AND ".join(pre_terms_role)
    pre_terms = " ; ".join(pre_terms_def + [pre_terms_role])

    return pre_text, pre_terms, return_type

def add_um(sentence):
    words = sentence.split()
    new_words = []
    mapping = {}
    offset = 0
    for i, word in enumerate(words):
        mapping[i] = len(new_words)
        new_words.append(word)
        if i > 0 and i < len(words) - 2 and random.random() > 0.5:
            num_um = random.choice([1,2,3])
            for j in range(num_um):
                new_words.append("um")
    return " ".join(new_words), mapping

def sprinkle(text, phi, _type):
    if "preposition" in _type:
        return text, phi, _type
    
    if random.random() >= sprinkle_prob:
        return text, phi, _type
    
    um_text, token_mapping = add_um(text)
    um_phi = []
    for t in phi.split():
        if t.isnumeric():
            um_phi += [str(token_mapping[int(t)])]
        else:
            um_phi += [t]
    um_phi = " ".join(um_phi)
    
    return um_text, um_phi, "sprinkle"
            
train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

proposing_prob = 0.05
sprinkle_prob = 0.05
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(lambda x: translate(*x), axis=1, result_type='expand')
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(lambda x: sprinkle(*x), axis=1, result_type='expand')
dev_df[['sentence', 'LF', 'type']] = dev_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
test_df[['sentence', 'LF', 'type']] = test_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
gen_df[['sentence', 'LF', 'type']] = gen_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')

In [6]:
dataset_postfix = "preposing+sprinkles"
train_df.to_csv(f'./cogs_{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./cogs_{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./cogs_{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./cogs_{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)

### ReCOGS (Number of resampling iterations = 5)
It seems like the performance gain from increasing the number of resampling iterations dimish quickly after getting the number above 10. We are trying 5 here.

In [7]:
set_seed(seed)
existing_digit_pool = set([])
# loading target vocab to random sample our variable names
for k, v in load_vocab("./model/tgt_vocab.txt").items():
    if k.isnumeric():
        existing_digit_pool.add(k)
existing_digit_pool = list(existing_digit_pool)

def translate(text, phi):
    
    if len(phi.split()) == 1:
        return text, phi
    elif "LAMBDA" in phi:
        phi_split = phi.split()
        if len(phi_split) == 7:
            return text, phi
        v_pos = []
        idx = 0
        for t in phi_split:
            if t == text:
                v_pos += [idx]
            idx += 1
        for p in v_pos:
            phi_split[p] = phi_split[p+2]
            phi_split[p+2] = text
        
        return text, " ".join(phi_split)
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                def_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['role']} . {d['pred']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            nmod_terms += [f"nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = nmod_terms + role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)
    
    # final step, remove biases
    current_digit_pool = set([])
    for t in terms.split():
        if t.isnumeric():
            current_digit_pool.add(t)
    current_digit_pool = list(current_digit_pool)
    random.shuffle(current_digit_pool)
    sample_random_digit = random.sample(existing_digit_pool, k=len(current_digit_pool))
    digit_mapping = dict(zip(current_digit_pool, sample_random_digit))
    
    new_terms = []
    for t in terms.split():
        if t == "_" or t == "x":
            continue
        if t.isnumeric():
            new_terms += [digit_mapping[t]]
        else:
            new_terms += [t]

    terms = " ".join(new_terms)
    return text, terms

In [8]:
sampled_n = 5
append_k = 3072

train_dfs = []
for i in range(sampled_n):
    train_df_i = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
    train_df_i[['sentence', 'LF']] = train_df_i[['sentence', 'LF']].apply(lambda x: translate(*x), axis=1, result_type='expand')
    train_dfs += [train_df_i]
    
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df[['sentence', 'LF']] = dev_df[['sentence', 'LF']].apply(lambda x: translate(*x), axis=1, result_type='expand')
test_df[['sentence', 'LF']] = test_df[['sentence', 'LF']].apply(lambda x: translate(*x), axis=1, result_type='expand')
gen_df[['sentence', 'LF']] = gen_df[['sentence', 'LF']].apply(lambda x: translate(*x), axis=1, result_type='expand')

In [9]:
def reindex(LFs, existing_digit_pool):
    curr_digit = set([])
    for i in range(len(LFs)):
        for item in LFs[i].split():
            if item.isnumeric():
                curr_digit.add((i, int(item)))
    sampled_digits = random.sample(existing_digit_pool, k=len(curr_digit))
    digit_map = {}
    idx = 0
    for d in list(curr_digit):
        digit_map[d] = sampled_digits[idx]
        idx += 1
    
    reindex_LFs = []
    for i in range(len(LFs)):
        new_LFs = []
        for item in LFs[i].split():
            if item.isnumeric():
                new_LFs += [digit_map[(i, int(item))]]
            else:
                new_LFs += [item]
        reindex_LFs += [" ".join(new_LFs)]
        
    new_LF_prefix = []
    new_LF_body_nmod = []
    new_LF_body_verb = []
        
    for i in range(len(reindex_LFs)):
        new_LF_prefix.extend(reindex_LFs[i].split(" ; ")[:-1])
        for term in reindex_LFs[i].split(" ; ")[-1].split(" AND "):
            if "nmod" in term:
                new_LF_body_nmod += [term]
            else:
                new_LF_body_verb += [term]
                
    new_LF_body = new_LF_body_nmod + new_LF_body_verb
        
    return " ; ".join(new_LF_prefix) + " ; " + " AND ".join(new_LF_body)

start_indexes = [i*6 for i in range(append_k)]
append_data = []

for i in range(sampled_n):
    train_df_sorted = train_dfs[i].sort_values(by="sentence", key=lambda x: x.str.len())
    for start_index in start_indexes:
        conj_1 = train_df_sorted.iloc[-2-start_index].sentence
        if conj_1.split()[0] in {'The', 'A'}:
            conj_1_first = conj_1[0].lower()
        else:
            conj_1_first = conj_1[0]
            
        conj_2 = train_df_sorted.iloc[-3-start_index].sentence
        if conj_2.split()[0] in {'The', 'A'}:
            conj_2_first = conj_2[0].lower()
        else:
            conj_2_first = conj_2[0]
            
        append_data += [
            [train_df_sorted.iloc[-1-start_index].sentence[:-1]+\
            conj_1_first+\
            train_df_sorted.iloc[-2-start_index].sentence[1:-1]+\
            conj_2_first+\
            train_df_sorted.iloc[-3-start_index].sentence[1:],
            reindex(
                [
                    train_df_sorted.iloc[-1-start_index].LF,
                    train_df_sorted.iloc[-2-start_index].LF,
                    train_df_sorted.iloc[-3-start_index].LF
                ], existing_digit_pool
            ),
            'length_ood']
        ]
append_data = pd.DataFrame(append_data, columns =['sentence', 'LF', 'type'])

In [10]:
train_df = pd.concat(train_dfs)
train_df = pd.concat([train_df, append_data])

dataset_postfix = "recogs"
train_df.to_csv(f'./{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)

### Variable-free format
This is from Qiu et. al., 2022 paper

In [34]:
def cogs_lf_to_funcall(lf):
    """Converts the given COGS logical form into the variable-free form.
    - Nouns (entities and unaries) become values:
      Jack --> Jack
      cat ( x _ 1 ) --> cat
      * cat ( x _ 1 ) --> * cat
    - Verbs become functions, and their roles become argument names:
      eat . agent ( x _ 2 , Jack ) --> eat ( agent = Jack )
    - The variables representing nouns resolve to their values:
      cat ( x _ 1 ) AND eat . agent ( x _ 2 , x _ 1 ) --> eat ( agent = cat )
    This converter constructs a graph where variables are nodes and binaries
    are edges. After identifying the root, it then performs depth-first traversal
    to construct the output.
    Args:
    lf: Logical form string.
    Returns:
    The converted logical form.
    """
    if "LAMBDA" in lf or "(" not in lf:
        return lf

    # Parse the terms in the logical form
    # Example: toss . agent ( x _ 1 , John ) --> [toss, agent], [x _ 1, John]
    terms = []
    for raw_term in re.split(" ; | AND ", lf):
        match = re.match(r"(.*) \( (.*) \)", raw_term)
        if not match:
            raise ValueError(f"Malformed term: {raw_term}")
        labels = match.group(1).split(" . ")
        args = match.group(2).split(" , ")
        if len(args) not in (1, 2):
            raise ValueError(f"Invalid number of args: {args}")
        terms.append((labels, args))

    # `nodes` maps variables to node name (e.g., "x _ 3" -> "* cat").
    nodes = {}
    for labels, args in terms:
        if args[0] in nodes:
            # The variable has already been seen; check for conflicts.
            if nodes[args[0]] not in (labels[0], "* " + labels[0]):
                raise ValueError(
                    f"Conflicting node name: {nodes[args[0]]} vs. {labels[0]}")
        else:
            nodes[args[0]] = labels[0]

    # `children` maps variables to a list of (edge name, target node).
    children = {}
    # Potential root nodes; any node being a child will be removed.
    root_candidates = list(nodes)
    for labels, args in terms:
        if len(args) == 2:
            if args[0] not in children:
                children[args[0]] = []
            children[args[0]].append((" . ".join(labels[1:]), args[1]))
            if args[1] in root_candidates:
                root_candidates.remove(args[1])
    if len(root_candidates) != 1:
        raise ValueError(f"Multiple roots: {root_candidates}")
    root = root_candidates[0]

    # Depth-first traverse the graph to construct the funcall
    def dfs(node):
        if node not in nodes:
            # Named entity such as "John"
            if node.startswith("x _"):
                raise ValueError(f"Unbound variable {node}")
            if node in children:
                raise ValueError(f"Named entity {node} has children {children[node]}")
            return [node]
        else:
            # A noun like "cat" or a verb like "jump"
            if node not in children:
                return [nodes[node]]
            funcall_args = []
            for edge_label, edge_target in children[node]:
                funcall_args.append([edge_label, "="] + dfs(edge_target))
            funcall = [nodes[node], "("]
            for i, funcall_arg in enumerate(funcall_args):
                if i != 0:
                    funcall.append(",")
                funcall.extend(funcall_arg)
            funcall.append(")")
            return funcall

    return " ".join(dfs(root))

In [35]:
train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

train_df['LF'] = train_df[['LF']].apply(lambda x: cogs_lf_to_funcall(*x), axis=1,)
dev_df['LF'] = dev_df[['LF']].apply(lambda x: cogs_lf_to_funcall(*x), axis=1,)
test_df['LF'] = test_df[['LF']].apply(lambda x: cogs_lf_to_funcall(*x), axis=1,)
gen_df['LF'] = gen_df[['LF']].apply(lambda x: cogs_lf_to_funcall(*x), axis=1,)

In [36]:
dataset_postfix = "variable_free"
train_df.to_csv(f'./{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)

### Participial Verb Phrases

In [19]:
set_seed(seed)

noun_pool = set()
v_pool = set()
def collect_nv(text, phi, _type):
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    role_set = set([])
    for d in data:
        if d['type'] == 'role':
            v_pool.add(d['pred'])
        elif d['type'] == 'np':
            noun_pool.add(d['pred'])
            
    return text, phi, _type

def translate_regular(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)

    return text, terms, _type

def translate(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    role_set = set([])
    object_entvar = None
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
            if d['role'] == "theme":
                object_entvar = d['entvar']
                
            role_set.add(d['role'])
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    start_with_pp = False if text_split[0] in {'A', 'The'} else True
    
    if start_with_pp or (object_entvar is not None and "x _ " not in object_entvar):
        # combine
        def_terms = " ; ".join(def_terms)
        if def_terms == "":
            terms = " AND ".join(rest_terms)
        elif " AND ".join(rest_terms) == "":
            terms = def_terms
        else:
            terms = def_terms + " ; " + " AND ".join(rest_terms)
        return text, terms, _type
    
    if random.random() >= participial_prob:
        # combine
        def_terms = " ; ".join(def_terms)
        if def_terms == "":
            terms = " AND ".join(rest_terms)
        elif " AND ".join(rest_terms) == "":
            terms = def_terms
        else:
            terms = def_terms + " ; " + " AND ".join(rest_terms)
        return text, terms, _type
    
    # agent -> subject
    # theme -> object
    if "theme" in phi:
        # only one object if it exists!
        assert 1 == phi.split().count("theme")
        modify_subject = True if random.random() < 0.5 else False
    else:
        modify_subject = True
    
    # reindex!
    def reindex_terms(terms, offset=1, distance=3):
        new_terms = []
        for term in terms:
            new_term = []
            for t in term.split():
                if t.isnumeric() and int(t) > offset:
                    new_term += [str(int(t)+distance)]
                else:
                    new_term += [t]
            new_terms += [" ".join(new_term)]
        return new_terms
    
    # we need to decide whether to add to subj or obj.
    second_text_pp = False
    if modify_subject:
        subject = text_split[0] if start_with_pp else text_split[1]
        verb_pp = random.choice(list(verb_pp_map.keys()))
        noun_pp = random.choice(list(noun_pool))
        articles = 'a' if random.random() < 0.5 else 'the'
        text_pp = [verb_pp_map[verb_pp], articles, noun_pp]
        if articles == 'a':
            pp_np_term =  [f"{noun_pp} ( x _ 4 )"]
        elif articles == 'the':
            pp_np_term = [f"* {noun_pp} ( x _ 4 )"]
        pp_role_term = [f"{subject} . acl . {verb_pp} ( x _ 1 , x _ 4 )"]
        
        if random.random() < 0.5:
            second_text_pp = True
            second_verb_pp = random.choice(list(verb_pp_map.keys()))
            second_noun_pp = random.choice(list(noun_pool))
            second_articles = 'a' if random.random() < 0.5 else 'the'
            text_pp += [verb_pp_map[second_verb_pp], second_articles, second_noun_pp]
            if second_articles == 'a':
                pp_np_term += [f"{second_noun_pp} ( x _ 7 )"]
            elif second_articles == 'the':
                pp_np_term += [f"* {second_noun_pp} ( x _ 7 )"]
            pp_role_term += [f"{noun_pp} . acl . {second_verb_pp} ( x _ 4 , x _ 7 )"]
            
        def_terms = reindex_terms(def_terms, distance=6 if second_text_pp else 3)
        rest_terms = reindex_terms(rest_terms, distance=6 if second_text_pp else 3)
        
        object_pos = 1
        
    else:
        # locate the obj
        assert "x _ " in object_entvar
        object_pos = int(object_entvar.split()[-1])
        _object = text_split[object_pos]
        
        verb_pp = random.choice(list(verb_pp_map.keys()))
        noun_pp = random.choice(list(noun_pool))
        articles = 'a' if random.random() < 0.5 else 'the'
        text_pp = [verb_pp_map[verb_pp], articles, noun_pp]
        if articles == 'a':
            pp_np_term = [f"{noun_pp} ( x _ {object_pos+3} )"]
        elif articles == 'the':
            pp_np_term = [f"* {noun_pp} ( x _ {object_pos+3} )"]
        pp_role_term = [f"{_object} . acl . {verb_pp} ( x _ {object_pos} , x _ {object_pos+3} )"]
        
        if random.random() < 0.5:
            second_text_pp = True
            second_verb_pp = random.choice(list(verb_pp_map.keys()))
            second_noun_pp = random.choice(list(noun_pool))
            second_articles = 'a' if random.random() < 0.5 else 'the'
            text_pp += [verb_pp_map[second_verb_pp], second_articles, second_noun_pp]
            if second_articles == 'a':
                pp_np_term += [f"{second_noun_pp} ( x _ {object_pos+6} )"]
            elif second_articles == 'the':
                pp_np_term += [f"* {second_noun_pp} ( x _ {object_pos+6} )"]
            pp_role_term += [f"{noun_pp} . acl . {second_verb_pp} ( x _ {object_pos+3} , x _ {object_pos+6} )"]
 
        def_terms = reindex_terms(def_terms, object_pos, distance=6 if second_text_pp else 3)
        rest_terms = reindex_terms(rest_terms, object_pos, distance=6 if second_text_pp else 3)
    
    for np_t in pp_np_term:
        if np_t[0] == "*":
            def_terms += [np_t]
        else:
            rest_terms += [np_t]
    rest_terms += pp_role_term
    text_split = text_split[:object_pos+1] + text_pp + text_split[object_pos+1:]
    new_text = " ".join(text_split)
    def_terms.sort(key = lambda x: int(x.split()[-2]))   
    rest_terms.sort(
        key = lambda x: (
            int(x.split()[-7]) if x.split()[-1] == "acl" else int(x.split()[-2]) if x.split()[-5] == "(" else int(x.split()[-6]) if x.split()[-2].isnumeric() else int(x.split()[-4]), 
            int(x.split()[-3]) if x.split()[-1] == "acl" else -1 if x.split()[-5] == "(" else int(x.split()[-2]) if x.split()[-2].isnumeric() else new_text.split().index(x.split()[-2])
        )
    )  
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)
    
    if second_text_pp:
        return new_text, terms, "subj_participle_verb_phrase_nested" if modify_subject else "obj_participle_verb_phrase_nested"
    else:
        return new_text, terms, "subj_participle_verb_phrase" if modify_subject else "obj_participle_verb_phrase"
    

In [22]:
participial_prob = 0.15
train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

with open('./verb_pp_map.json', 'r') as openfile:
    verb_pp_map = json.load(openfile)
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(
    lambda x: collect_nv(*x), axis=1, result_type='expand'
)
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(
    lambda x: translate(*x), axis=1, result_type='expand'
)

dev_df[['sentence', 'LF', 'type']] = dev_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
test_df[['sentence', 'LF', 'type']] = test_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
gen_df[['sentence', 'LF', 'type']] = gen_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')

In [23]:
dataset_postfix = "participle_verb"
train_df.to_csv(f'./cogs_{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./cogs_{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./cogs_{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./cogs_{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)

### Participial Verb Phrases (Easy)

In [36]:
set_seed(seed)

noun_pool = set()
v_pool = set()
def collect_nv(text, phi, _type):
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    role_set = set([])
    for d in data:
        if d['type'] == 'role':
            v_pool.add(d['pred'])
        elif d['type'] == 'np':
            noun_pool.add(d['pred'])
            
    return text, phi, _type

def translate_regular(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)

    return text, terms, _type

def translate(text, phi, _type):
    
    if len(phi.split()) == 1:
        return text, phi, _type
    elif "LAMBDA" in phi:
        return text, phi, _type
    
    # parse
    text_split = text.split()
    data = []    
    conjs = re.split(r"\s*(?:AND|;)\s*", phi)
    for conj in conjs: 
        if np_re.search(conj):
            d = parse_np(conj)
        elif pred_re.search(conj):
            d = parse_pred(conj)
        elif mod_re.search(conj):
            d = parse_mod(conj)
        else:
            raise Exception(f"Conjunct could not be parsed: {conj}")
        data.append(d)
    
    # collect
    def_terms = []
    role_terms = []
    nmod_terms = []
    role_set = set([])
    object_entvar = None
    for d in data:
        if d['type'] == 'np':
            if d['definiteness'] == '*':
                def_terms += [f"* {d['pred']} ( {d['entvar']} )"]
            else:
                role_terms += [f"{d['pred']} ( {d['entvar']} )"]
        if d['type'] == 'role':
            role_terms += [f"{d['pred']} . {d['role']} ( {d['eventvar']} , {d['entvar']} )"]
            if d['role'] == "theme":
                object_entvar = d['entvar']
                
            role_set.add(d['role'])
        elif d['type'] == 'mod':
            assert "x _" in d['e1']
            role_terms += [f"{d['nppred']} . nmod . {d['pred']} ( {d['e1']} , {d['e2']} )"]
            
    # sort def_terms
    def_terms = [*set(def_terms)]
    def_terms.sort(key = lambda x: int(x.split()[-2]))    
    rest_terms = role_terms
    
    start_with_pp = False if text_split[0] in {'A', 'The'} else True
    
    if start_with_pp or (object_entvar is not None and "x _ " not in object_entvar):
        # combine
        def_terms = " ; ".join(def_terms)
        if def_terms == "":
            terms = " AND ".join(rest_terms)
        elif " AND ".join(rest_terms) == "":
            terms = def_terms
        else:
            terms = def_terms + " ; " + " AND ".join(rest_terms)
        return text, terms, _type
    
    if random.random() >= participial_prob:
        # combine
        def_terms = " ; ".join(def_terms)
        if def_terms == "":
            terms = " AND ".join(rest_terms)
        elif " AND ".join(rest_terms) == "":
            terms = def_terms
        else:
            terms = def_terms + " ; " + " AND ".join(rest_terms)
        return text, terms, _type
    
    # agent -> subject
    # theme -> object
    if "theme" in phi:
        # only one object if it exists!
        assert 1 == phi.split().count("theme")
        modify_subject = True if random.random() < 0.5 else False
    else:
        modify_subject = True
    
    # reindex!
    def reindex_terms(terms, offset=1, distance=3):
        new_terms = []
        for term in terms:
            new_term = []
            for t in term.split():
                if t.isnumeric() and int(t) > offset:
                    new_term += [str(int(t)+distance)]
                else:
                    new_term += [t]
            new_terms += [" ".join(new_term)]
        return new_terms
    
    # we need to decide whether to add to subj or obj.
    second_text_pp = False
    if modify_subject:
        subject = text_split[0] if start_with_pp else text_split[1]
        verb_pp = random.choice(list(verb_pp_map.keys()))
        noun_pp = random.choice(list(noun_pool))
        articles = 'a' if random.random() < 0.5 else 'the'
        text_pp = [verb_pp_map[verb_pp], articles, noun_pp]
        if articles == 'a':
            pp_np_term =  [f"{noun_pp} ( x _ 4 )"]
        elif articles == 'the':
            pp_np_term = [f"* {noun_pp} ( x _ 4 )"]
        pp_role_term = [f"{subject} . nmod . {verb_pp} ( x _ 1 , x _ 4 )"]
        
        if random.random() < 0.5:
            second_text_pp = True
            second_verb_pp = random.choice(list(verb_pp_map.keys()))
            second_noun_pp = random.choice(list(noun_pool))
            second_articles = 'a' if random.random() < 0.5 else 'the'
            text_pp += [verb_pp_map[second_verb_pp], second_articles, second_noun_pp]
            if second_articles == 'a':
                pp_np_term += [f"{second_noun_pp} ( x _ 7 )"]
            elif second_articles == 'the':
                pp_np_term += [f"* {second_noun_pp} ( x _ 7 )"]
            pp_role_term += [f"{noun_pp} . nmod . {second_verb_pp} ( x _ 4 , x _ 7 )"]
            
        def_terms = reindex_terms(def_terms, distance=6 if second_text_pp else 3)
        rest_terms = reindex_terms(rest_terms, distance=6 if second_text_pp else 3)
        
        object_pos = 1
        
    else:
        # locate the obj
        assert "x _ " in object_entvar
        object_pos = int(object_entvar.split()[-1])
        _object = text_split[object_pos]
        
        verb_pp = random.choice(list(verb_pp_map.keys()))
        noun_pp = random.choice(list(noun_pool))
        articles = 'a' if random.random() < 0.5 else 'the'
        text_pp = [verb_pp_map[verb_pp], articles, noun_pp]
        if articles == 'a':
            pp_np_term = [f"{noun_pp} ( x _ {object_pos+3} )"]
        elif articles == 'the':
            pp_np_term = [f"* {noun_pp} ( x _ {object_pos+3} )"]
        pp_role_term = [f"{_object} . nmod . {verb_pp} ( x _ {object_pos} , x _ {object_pos+3} )"]
        
        if random.random() < 0.5:
            second_text_pp = True
            second_verb_pp = random.choice(list(verb_pp_map.keys()))
            second_noun_pp = random.choice(list(noun_pool))
            second_articles = 'a' if random.random() < 0.5 else 'the'
            text_pp += [verb_pp_map[second_verb_pp], second_articles, second_noun_pp]
            if second_articles == 'a':
                pp_np_term += [f"{second_noun_pp} ( x _ {object_pos+6} )"]
            elif second_articles == 'the':
                pp_np_term += [f"* {second_noun_pp} ( x _ {object_pos+6} )"]
            pp_role_term += [f"{noun_pp} . nmod . {second_verb_pp} ( x _ {object_pos+3} , x _ {object_pos+6} )"]
 
        def_terms = reindex_terms(def_terms, object_pos, distance=6 if second_text_pp else 3)
        rest_terms = reindex_terms(rest_terms, object_pos, distance=6 if second_text_pp else 3)
    
    for np_t in pp_np_term:
        if np_t[0] == "*":
            def_terms += [np_t]
        else:
            rest_terms += [np_t]
    rest_terms += pp_role_term
    text_split = text_split[:object_pos+1] + text_pp + text_split[object_pos+1:]
    new_text = " ".join(text_split)
    def_terms.sort(key = lambda x: int(x.split()[-2]))   
    rest_terms.sort(
        key = lambda x: (
            int(x.split()[-7]) if x.split()[-1] == "nmod" else int(x.split()[-2]) if x.split()[-5] == "(" else int(x.split()[-6]) if x.split()[-2].isnumeric() else int(x.split()[-4]), 
            int(x.split()[-3]) if x.split()[-1] == "nmod" else -1 if x.split()[-5] == "(" else int(x.split()[-2]) if x.split()[-2].isnumeric() else new_text.split().index(x.split()[-2])
        )
    )  
    # combine
    def_terms = " ; ".join(def_terms)
    if def_terms == "":
        terms = " AND ".join(rest_terms)
    elif " AND ".join(rest_terms) == "":
        terms = def_terms
    else:
        terms = def_terms + " ; " + " AND ".join(rest_terms)
    
    if second_text_pp:
        return new_text, terms, "subj_participle_verb_phrase_nested" if modify_subject else "obj_participle_verb_phrase_nested"
    else:
        return new_text, terms, "subj_participle_verb_phrase" if modify_subject else "obj_participle_verb_phrase"
    

In [37]:
participial_prob = 0.15
train_df = pd.read_csv("./cogs/train.tsv", sep="\t", names=['sentence', 'LF', 'type'])
dev_df = pd.read_csv("./cogs/dev.tsv", sep="\t", names=['sentence', 'LF', 'type'])
test_df = pd.read_csv("./cogs/test.tsv", sep="\t", names=['sentence', 'LF', 'type'])
gen_df = pd.read_csv("./cogs/gen.tsv", sep="\t", names=['sentence', 'LF', 'type'])

with open('./verb_pp_map.json', 'r') as openfile:
    verb_pp_map = json.load(openfile)
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(
    lambda x: collect_nv(*x), axis=1, result_type='expand'
)
train_df[['sentence', 'LF', 'type']] = train_df[['sentence', 'LF', 'type']].apply(
    lambda x: translate(*x), axis=1, result_type='expand'
)

dev_df[['sentence', 'LF', 'type']] = dev_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
test_df[['sentence', 'LF', 'type']] = test_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')
gen_df[['sentence', 'LF', 'type']] = gen_df[['sentence', 'LF', 'type']].apply(lambda x: translate_regular(*x), axis=1, result_type='expand')

In [38]:
dataset_postfix = "participle_verb_easy"
train_df.to_csv(f'./cogs_{dataset_postfix}/train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(f'./cogs_{dataset_postfix}/dev.tsv', sep='\t', index=False, header=False)
test_df.to_csv(f'./cogs_{dataset_postfix}/test.tsv', sep='\t', index=False, header=False)
gen_df.to_csv(f'./cogs_{dataset_postfix}/gen.tsv', sep='\t', index=False, header=False)